### Load data

In [3]:
from sys import platform
if platform == 'linux' or platform == 'linux2':
    datapath = '/home/alin/MyLearning/Kaggle/Sberbank/data/'
else:
    datapath = 'C:/Users/alin/Documents/SelfStudy/MyLearning/Kaggle/Sberbank/data/'
print datapath

/home/alin/MyLearning/Kaggle/Sberbank/data/


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import seaborn as sns
sns.set(color_codes=True)
from sklearn import metrics
from sklearn.preprocessing import scale 

from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor

In [5]:

train_df = pd.read_csv(datapath + 'train.csv')
# change column nams to lower case
train_df.columns = train_df.columns.str.lower()
original_cols = train_df.columns.tolist()
test_df = pd.read_csv(datapath + 'test.csv')
test_df.columns = test_df.columns.str.lower()

macro_df = pd.read_csv(datapath + 'macro.csv')
macro_df.columns = macro_df.columns.str.lower()


In [6]:
%qtconsole

In [8]:
train_df = pd.merge(train_df, macro_df, on = 'timestamp')
test_df = pd.merge(test_df, macro_df, on = 'timestamp')

### process timestamp and build_year fields

In [13]:
 from datetime import datetime

In [14]:
train_df['year'] = map(lambda x: datetime.strptime(x, '%Y-%m-%d').year, train_df.timestamp)
test_df['year'] = map(lambda x: datetime.strptime(x, '%Y-%m-%d').year, test_df.timestamp)

#train_df['year'] = map(lambda x: int(x[0:4]), train_df.timestamp)
#train_df['month'] = map(lambda x: int(x[5:7]), train_df.timestamp)


In [15]:
train_df['month'] = map(lambda x: datetime.strptime(x, '%Y-%m-%d').month, train_df.timestamp)
test_df['month'] = map(lambda x: datetime.strptime(x, '%Y-%m-%d').month, test_df.timestamp)

In [16]:


del train_df['timestamp']
del test_df['timestamp']

### some  values are like '90,80" etc.

In [17]:
def myaoti(x):
    try:
        return float(str(x).replace(',', ''))
    except:
        return np.nan

In [18]:
#train_df['modern_education_share'] = train_df.modern_education_share.map(lambda x: float(str(x).replace(',','')))
train_df['modern_education_share'] = train_df.modern_education_share.map(myaoti)

train_df['old_education_build_share'] = train_df.old_education_build_share.map(myaoti)

train_df['child_on_acc_pre_school'] = train_df.child_on_acc_pre_school.map(myaoti)

test_df['modern_education_share'] = test_df.modern_education_share.map(myaoti)

test_df['old_education_build_share'] = test_df.old_education_build_share.map(myaoti)

test_df['child_on_acc_pre_school'] = test_df.child_on_acc_pre_school.map(myaoti)



In [19]:
def bi_convert(df, feature, pos = 'yes'):
    df[feature] = df[feature].map(lambda x: 1 if x == pos else 0)

In [20]:

bi_convert(train_df, 'thermal_power_plant_raion')
bi_convert(train_df, 'incineration_raion')
bi_convert(train_df, 'oil_chemistry_raion')
bi_convert(train_df, 'radiation_raion')
bi_convert(train_df, 'railroad_terminal_raion')
bi_convert(train_df, 'big_market_raion')
bi_convert(train_df, 'nuclear_reactor_raion')
bi_convert(train_df, 'detention_facility_raion')
bi_convert(train_df, 'water_1line')
bi_convert(train_df, 'big_road1_1line')
bi_convert(train_df, 'railroad_1line')
bi_convert(train_df, 'product_type', 'OwnerOccupier')
bi_convert(train_df, 'culture_objects_top_25')

bi_convert(test_df, 'thermal_power_plant_raion')
bi_convert(test_df, 'incineration_raion')
bi_convert(test_df, 'oil_chemistry_raion')
bi_convert(test_df, 'radiation_raion')
bi_convert(test_df, 'railroad_terminal_raion')
bi_convert(test_df, 'big_market_raion')
bi_convert(test_df, 'nuclear_reactor_raion')
bi_convert(test_df, 'detention_facility_raion')
bi_convert(test_df, 'water_1line')
bi_convert(test_df, 'big_road1_1line')
bi_convert(test_df, 'railroad_1line')
bi_convert(test_df, 'product_type', 'OwnerOccupier')
bi_convert(test_df, 'culture_objects_top_25')


In [21]:
def ecology_convert(x):
    eco ={'poor':-1, 'no data': 0, 'good': 1, 'satisfactory': 2, 'excellent': 3 }
    return eco[x]

In [22]:
train_df['ecology'] = train_df.ecology.map(ecology_convert)
test_df['ecology'] = test_df.ecology.map(ecology_convert)



In [23]:
train_df['sub_area'] = pd.Categorical(train_df.sub_area).codes
test_df['sub_area'] = pd.Categorical(test_df.sub_area).codes

In [28]:
y_train = np.log(train_df.price_doc.as_matrix()+1)


In [ ]:
train_df.drop(['id', 'price_doc'], axis = 1, inplace = True )

In [31]:
tid = test_df.id.copy()

In [34]:
test_df.drop(['id'], axis = 1, inplace = True)

### check out the predictive power of each variable
for numerical variable, use linear regression or tree(?)

In [38]:
def rmsle(lp, la):
    return np.sqrt(np.mean(np.square(lp - la)))

In [39]:
from sklearn.preprocessing import Imputer
impute = Imputer()

In [42]:
# a decision tree regresson on a simple numerical variable with performance on the validation set
def single_num(feature):
    try:
        X_train = train0[feature].as_matrix().reshape(-1,1)
        X_valid = val0[feature].as_matrix().reshape(-1,1)
        X_tr = impute.fit_transform(X_train)
        X_val = impute.fit_transform(X_valid)
        tree = DecisionTreeRegressor(max_depth=5)
        tree.fit(X_tr, y_train0)
        y_pred = tree.predict(X_val)
        return rmsle(y_pred, y_val0)
    except:
        return 1

### order features by their single predicting power

In [43]:
power = []
num_col = train0.columns.tolist()
for col in num_col:
   
    power.append((col, single_num(col)))
#power.append(('sub_area', single_cat()))


In [44]:
power = sorted(power, key = lambda x: x[1])

In [46]:
used_features = [f for (f,s) in power[0:50]]

### try xgboost

In [49]:
import xgboost as xgb

In [50]:
X_train = train_df[used_features].values
X_test = test_df[used_features].values

In [59]:
xgtrain = xgb.DMatrix(X_train, y_train)
xgtest = xgb.DMatrix(X_test)

In [52]:
xgb_params = {
    'eta': 0.1,
    'max_depth': 5,
     'min_child_weight': 1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

In [55]:
#cvresult = xgb.cv(xgb_params, xgtrain, num_boost_round=1000, nfold=5, metrics='rmse', early_stopping_rounds=50, verbose_eval=True)

In [57]:
num_rounds = 193

In [60]:
model = xgb.train(xgb_params, xgtrain, num_boost_round=num_rounds, evals = [(xgtrain, 'train')], verbose_eval=20)
# make prediction
y_pred1 = model.predict(xgtest)


[0]	train-rmse:13.6116
[20]	train-rmse:1.72302
[40]	train-rmse:0.503307
[60]	train-rmse:0.452667
[80]	train-rmse:0.444803
[100]	train-rmse:0.43858
[120]	train-rmse:0.433512
[140]	train-rmse:0.428487
[160]	train-rmse:0.423099
[180]	train-rmse:0.41785
[192]	train-rmse:0.41493


In [83]:
price1 = np.exp(y_pred1) - 1

In [88]:
output1 = pd.DataFrame({'id':tid, 'price_doc': price1})

In [89]:
output1.to_csv('/home/alin/MyLearning/Kaggle/Sberbank/data/submission1.csv', index = False)

In [68]:
import lightgbm as lgb

In [69]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_test = lgb.Dataset(X_test)

In [70]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression_l2',
    'metric': {'rmse'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
    }


In [74]:
#cv_lgb = lgb.cv(params, lgb_train, num_boost_round = 1000, nfold=5, early_stopping_rounds = 50, 
#                verbose_eval = True)

In [75]:
gbm = lgb.train(params,lgb_train,num_boost_round=300)

In [76]:
y_pred2 = gbm.predict(X_test)

In [90]:
price2 = np.exp(y_pred2) - 1

In [91]:
output2 = pd.DataFrame({'id':tid, 'price_doc': price2})
output2.to_csv('/home/alin/MyLearning/Kaggle/Sberbank/data/submission2.csv', index = False)